## **Postulación Sunai: Python Developer JR**

In [116]:
# Load packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')
%matplotlib qt

In [117]:
#Set working directory
# path_in = input('Enter path to working directory: ')
path_in = r'C:\Users\fespi\OneDrive\Desktop\data_sunai'

files = [os.path.join(path_in, f) for f in os.listdir(path_in) if f.endswith('.xlsx')]
plants = [files[i][files[i].rfind('\\')+6:-5] for i in range(len(files))]

In [178]:
# Read data and transform to dataframe
def data_sunai(files):
    df = pd.read_excel(files, header=0, parse_dates=True, na_filter=True)
    df.replace('data_faltante',0, inplace=True)
    df['date'], df['time'] = zip(*[(d.date(), d.time()) for d in df['fecha_im']])
    df['date'], df['time'] = df['date'].astype(str), df['time'].astype(str)
    cols = list(df.columns.values)
    df = df[cols[0:2] + cols[-2:] + cols[2:-2]]
    df = df[df.columns[[0,2,3,14]]]
    
    df_pv = df.pivot(index='time', columns='id_i', values='active_power_im')  # Create a pivot table
    df_pv.rename_axis(None, axis=1, inplace=True)
    df_pv['date'] = df['date'][0]
    return df_pv

def data_inv_sunai(df_pv):
    df_pv2 = df_pv[df_pv.columns[:-1]]
    df_inv = pd.DataFrame(index = ['sum', 'min', 'max'], columns = df_pv2.columns,
                      data = [df_pv2.sum().values, df_pv2.min().values, df_pv2.max().values]).T
    df_inv.fillna(0, inplace=True)  # To replace NaN values with 0
    df_inv.index.name = 'id_i'
    df_inv['date'] = df_pv['date'][0]
    return df_inv

def plot_data_sunai(df_pv, plants):
    fig, ax = plt.subplots(figsize=(8,5), dpi=100)
    name_file_img = 'act_power_im_inv'

    df_pv.plot(kind='line', rot=30, ax=ax, lw = 1.6, alpha=.8, legend=True)
    ax.set(xlabel="Time. Date: "+str(df_pv['date'][0]), ylabel='active_power_im (p.u.)', 
                title='Active power import by inverter')

    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[0:], labels=labels[0:])
    plt.tight_layout();  # plt.show()
    fig.savefig(path_in+'\\'+name_file_img+'_'+plants+'.png', bbox_inches='tight', dpi=150)
    print('Graph saved in: ' + path_in+'\\'+name_file_img+'_'+plants+'.png')
    
def plot_data_inv(df_inv):
    fig, ax = plt.subplots(figsize=(8,5), dpi=100)
    labs1st = ['Sum of active power', 'Min of active_power', 'Max of active power']

    df_inv.plot(y = df_inv.columns[0], kind="bar", rot=30, ax=ax, width=.4, alpha=.8, legend=False)
    ax.set(xlabel="Inverter", ylabel='active_power_im (p.u.)', 
           title=str(labs1st[0])+' by inverter. Date: '+str(df_inv['date'].iloc[0]))
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    ax.grid(axis='y', alpha=.5)
    plt.tight_layout();  plt.show()
    
def data_save(df_inv, plants):
    df_inv.to_csv(path_in + '\\' + plants + '_summarized.txt', sep='\t', encoding='utf-8')
    print('File saved in: ' + path_in + '\\' + plants + '_summarized.txt')

In [184]:
plt.close('all')  # Closing previous figures
data_sun, data_inv = [], []
for i in range(len(files)):
    print("Index:", i, "is plant:", plants[i])
    data_sun.append(data_sunai(files[i]))
    data_inv.append(data_inv_sunai(data_sun[i]))
    plot_data_sunai(data_sun[i], plants[i])
    plot_data_inv(data_inv[i])
    data_save(data_inv[i], plants[i])
    print("\n")
    if i == len(files)-1:
        print("All files processed")

Index: 0 is plant: plantas_python_1
Graph saved in: C:\Users\fespi\OneDrive\Desktop\data_sunai\act_power_im_inv_plantas_python_1.png
File saved in: C:\Users\fespi\OneDrive\Desktop\data_sunai\plantas_python_1_summarized.txt


Index: 1 is plant: plantas_python_2
Graph saved in: C:\Users\fespi\OneDrive\Desktop\data_sunai\act_power_im_inv_plantas_python_2.png
File saved in: C:\Users\fespi\OneDrive\Desktop\data_sunai\plantas_python_2_summarized.txt


All files processed
